In [1]:
from configuration import Configuration

cfg = Configuration.load(
    "../../../../../data/a0910/model/IRT/init/configuration.json"
)
print(cfg)

logger: <Logger IRT (INFO)>
model_name: IRT
root: G:\program\XCD
dataset: a0910
timestamp: 20201117204012
workspace: init
root_data_dir: G:\program\XCD\data\a0910
data_dir: G:\program\XCD\data\a0910\data
root_model_dir: G:\program\XCD\data\a0910\model\IRT
model_dir: G:\program\XCD\data\a0910\model\IRT\init
cfg_path: G:\program\XCD\data\a0910\model\IRT\init\configuration.json
begin_epoch: 0
end_epoch: 10
batch_size: 256
save_epoch: 1
optimizer: Adam
optimizer_params: {'learning_rate': 0.01, 'clip_gradient': 1}
lr_params: {'update_params': {}}
train_select: None
save_select: None
ctx: [cpu(0)]
toolbox_params: {}
hyper_params: {'user_num': 4163, 'item_num': 17746, 'theta_range': [-4, 4], 'a_range': [0.25, 2], 'b_range': [-2, 2], 'c_range': [0, 0.4]}
init_params: {}
loss_params: {}
caption: 
validation_result_file: G:\program\XCD\data\a0910\model\IRT\init\result.json


In [2]:
from sym import get_net

net = get_net(**cfg.hyper_params)
net

IRT(
  (theta): Embedding(4163 -> 1, float32)
  (b): Embedding(17746 -> 1, float32)
  (a): Embedding(17746 -> 1, float32)
  (c): Embedding(17746 -> 1, float32)
)

In [3]:
from longling.ML import get_epoch_params_filepath
from sym import net_init

epoch = 2
model_file = get_epoch_params_filepath(cfg.model_name, epoch, cfg.model_dir)
net_init(net, cfg, model_file=model_file, logger=cfg.logger)

IRT, INFO load params from existing model file G:\program\XCD\data\a0910\model\IRT\init\IRT-0002.parmas


In [4]:
from longling import path_append
from longling.ML.toolkit import result_format
from etl import etl
from sym import eval_f

test, test_df = etl(path_append(cfg.data_dir, "test.csv", to_str=True), params=cfg)
print(result_format(eval_f(net, test)))

IRT, INFO Starting to loading data from G:\program\XCD\data\a0910\data\test.csv
IRT, INFO Finished to loading data from G:\program\XCD\data\a0910\data\test.csv in 0.018976 seconds
evaluating: 100%|██████████| 218/218 [00:00<00:00, 258.02it/s]


           precision    recall        f1  support
0.0         0.817368  0.156265  0.262369    19275
1.0         0.687700  0.981554  0.808762    36485
macro_avg   0.752534  0.568909  0.535566    55760
accuracy: 0.696270	macro_auc: 0.739607	macro_aupoc: 0.826107


In [5]:
import pandas as pd
from XCD.IRT.metrics import eval

user_id = list(range(cfg.hyper_params["user_num"]))
theta = net.get_theta(user_id)
user_df = pd.DataFrame({
    "user_id": user_id,
    "theta": theta.asnumpy().tolist()
})
user_df.head()

,user_id,theta
0,0,0.616657
1,1,3.742713
2,2,0.880027
3,3,2.513194
4,4,1.953740


In [6]:
item_id = list(range(cfg.hyper_params["item_num"]))
a = net.get_a(item_id)
b = net.get_b(item_id)
c = net.get_c(item_id)
item_df = pd.DataFrame({
    "item_id": item_id,
    "a": a.asnumpy().tolist(),
    "b": b.asnumpy().tolist(),
    "c": c.asnumpy().tolist(),
})
item_df.head()

,item_id,a,b,c
0,0,0.712413,0.331391,0.4
1,1,1.524633,-0.843536,0.4
2,2,1.700163,-0.464836,0.4
3,3,1.644652,-0.475207,0.4
4,4,1.620100,-0.073687,0.4


In [7]:
eval(test_df, user_df, item_df)

           precision    recall        f1  support
0.0         0.817368  0.156265  0.262369    19275
1.0         0.687700  0.981554  0.808762    36485
macro_avg   0.752534  0.568909  0.535566    55760
accuracy: 0.696270	macro_auc: 0.739607	macro_aupoc: 0.826015
